<h1>TORONTO NEIGHBORHOOD DATA SCRAPING, PROCESSING AND CLUSTER MODEL</h1>

<h2>DATA SCRAPPING</h2>

<h6>Installing BeautifulSoup4</h6>

In [171]:
!pip install BeautifulSoup4

<h6>Importing libraries</h6>

In [172]:
import pandas as pd
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


In [173]:
website_url = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050').text

<h6>Using the BeautifulSoup package to transform the data in the table on the Wikipedia page into the above pandas dataframe¶</h6>

In [174]:
soup = BeautifulSoup(website_url,'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XnX4ZgpAMNEAA9O@OccAAADI","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":946188405,"wgRevisionId":945633050,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

<h6>By observation we can see that the tabular data is availabe in table and belongs to class="wikitable sortable"So let's extract only table</h6>

In [175]:
# create three lists to store table data
postalCode = []
borough = []
neighborhood = []

<h4>BEAUTIFUL SOUP USES</h4>
<p>soup.find('table').find_all('td')                   #finding table<br>
soup.find('table').find_all('tr')                      #finding all the rows of table<br>
for row in soup.find('table').find_all('tr'):          #for each row of table, finding all data<br>
    cells = row.find_all('td')<p>

In [176]:
# append data into respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells)>0):
        postalCode.append(cells[0].text)
        borough.append(cells[1].text)
        neighborhood.append(cells[2].text.rstrip('\n'))

In [177]:
#creating new dataframe

toronto_df = pd.DataFrame({"PostalCode": postalCode,
                          "Borough": borough,
                          "Neighborhood": neighborhood})
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<h2>DATA PROCESSING</h2>

In [178]:
# dropping cells with borough not assigned
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_drop.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [179]:
#grouping neighborhood in same borough
toronto_df_group = toronto_df_drop.groupby(["PostalCode","Borough"], as_index=False).agg(lambda x: ",".join(x))
toronto_df_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [180]:
#data processing for neighborhood column
for index, row in toronto_df_group.iterrows():
    if row["Neighborhood"]=="Not assigned":
        row["Neighborhood"]=row["Borough"]
toronto_df_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [181]:
toronto_df_group.shape

(103, 3)

<h6>Use the Geocoder package or the csv file to create dataframe with longitude and latitude values</h6>

In [182]:
df_lon_lat = pd.read_csv('Geospatial_Coordinates.csv')
df_lon_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [183]:
# rename the column "PostalCode"
df_lon_lat.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_lon_lat.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [184]:
# merge two table on the column "PostalCode"
toronto_df_new = toronto_df_group.merge(df_lon_lat, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h2>MODEL USING K-MEANS</h2>

<h6>Using geopy library to get the latitude and longitude values of Toronto</h6>

In [185]:
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [186]:
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'],toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='#8B0000',
    fill=True,
    fill_color='red',
    fill_opacity=0.7).add_to(map_toronto)
map_toronto

<h4>Integrating FourSquare API</h4>

In [187]:
CLIENT_ID = 'PVFOBPEDFVBJLLT0CDURISRX4G3UURD50QCYDR5DLYFUXYBM' # your Foursquare ID
CLIENT_SECRET = 'B2YBCXUJLCKYN0YKW15XDBBHHAB1BGROPGYTJQB4NPS1GZGV' # your Foursquare Secret
VERSION = '20191225' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PVFOBPEDFVBJLLT0CDURISRX4G3UURD50QCYDR5DLYFUXYBM
CLIENT_SECRET:B2YBCXUJLCKYN0YKW15XDBBHHAB1BGROPGYTJQB4NPS1GZGV


<h4>Now, let's get the top 100 venues that are within a radius of 500 meters.</h4>

In [188]:
# defining radius and limit of venues to get
radius=500
LIMIT=100

In [189]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, long in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [190]:
toronto_venues = getNearbyVenues(names=toronto_df_new['Neighborhood'],
                                   latitudes=toronto_df_new['Latitude'],
                                   longitudes=toronto_df_new['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [191]:
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.594054,Dimmi Bar & Trattoria,43.785277,-79.593144,Italian Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.594054,Nino D'Aversa Bakery Ltd,43.783782,-79.598828,Bakery
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.594054,Big Cannoli Lane,43.784973,-79.592910,Dessert Shop
3,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.594054,Market Lane Shopping Centre,43.785356,-79.593525,Shopping Mall
4,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.594054,Market Lane Pizzeria,43.785359,-79.593190,Italian Restaurant
5,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.594054,Cataldis.,43.785894,-79.592960,Grocery Store
6,"Guildwood,Morningside,West Hill",43.763573,-79.594054,On a Go bus,43.763810,-79.597458,Moving Target
7,Woburn,43.770992,-79.594054,Makeout Mountain,43.775081,-79.592159,Mountain
8,Cedarbrae,43.773136,-79.594054,Makeout Mountain,43.775081,-79.592159,Mountain
9,Scarborough Village,43.744734,-79.594054,Pizza Hut,43.743473,-79.595708,Pizza Place


In [192]:
toronto_venues.shape

(561, 7)

In [193]:
toronto_venues.groupby(["Neighborhood"]).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",5,5,5,5,5,5
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",2,2,2,2,2,2
"Alderwood,Long Branch",18,18,18,18,18,18
"Bathurst Manor,Downsview North,Wilson Heights",4,4,4,4,4,4
Bayview Village,5,5,5,5,5,5
"Bedford Park,Lawrence Manor East",4,4,4,4,4,4
"Birch Cliff,Cliffside West",4,4,4,4,4,4


In [194]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 87 uniques categories.


In [195]:
toronto_venues['Venue Category'].unique()[:50]

array(['Italian Restaurant', 'Bakery', 'Dessert Shop', 'Shopping Mall',
       'Grocery Store', 'Moving Target', 'Mountain', 'Pizza Place',
       'Sandwich Place', 'Discount Store', 'Coffee Shop', 'Hotel',
       'Rental Car Location', 'Afghan Restaurant', 'Racecourse',
       'Chinese Restaurant', 'Mediterranean Restaurant',
       'Paper / Office Supplies Store', 'Department Store',
       'Swiss Restaurant', 'Gym / Fitness Center', 'American Restaurant',
       'Bank', 'Gas Station', 'BBQ Joint', 'Convenience Store',
       'Golf Course', 'Construction & Landscaping', 'Dance Studio',
       'Skating Rink', 'Trail', 'Café', 'Food Stand', 'Bar', 'Gift Shop',
       'Vietnamese Restaurant', 'Ice Cream Shop', 'IT Services',
       'Video Store', 'Spa', 'Food Court', 'Bookstore', 'Drugstore',
       'Transportation Service', 'Electronics Store', 'Airport Terminal',
       "Men's Store", 'Restaurant', 'Steakhouse', 'Hobby Shop'],
      dtype=object)

<h4>Analysing Areas</h4>

In [196]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot.head()

,Afghan Restaurant,Airport Terminal,American Restaurant,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bookstore,Brewery,...,Taxi,Tea Room,Tennis Court,Thai Restaurant,Trail,Transportation Service,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Highland Creek,Rouge Hill,Port Union"
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Highland Creek,Rouge Hill,Port Union"
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Highland Creek,Rouge Hill,Port Union"
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Highland Creek,Rouge Hill,Port Union"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Highland Creek,Rouge Hill,Port Union"


In [197]:
toronto_onehot.shape

(561, 88)

In [198]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport Terminal,American Restaurant,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bookstore,...,Swiss Restaurant,Taxi,Tea Room,Tennis Court,Thai Restaurant,Trail,Transportation Service,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant
0,"Adelaide,King,Richmond",0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.4,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000
1,Agincourt,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000
4,"Alderwood,Long Branch",0.00,0.00,0.000000,0.055556,0.222222,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.055556
5,"Bathurst Manor,Downsview North,Wilson Heights",0.00,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.25,0.000000
6,Bayview Village,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.200000
7,"Bedford Park,Lawrence Manor East",0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000
8,"Birch Cliff,Cliffside West",0.00,0.00,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000
9,"Brockton,Exhibition Place,Parkdale Village",0.00,0.00,0.000000,0.000000,0.200000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000


<h4>Printing each neighborhood along with the top 5 most common venues</h4>

In [199]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
               venue  freq
0     Baseball Field   0.4
1          Racetrack   0.2
2             Garden   0.2
3        Golf Course   0.2
4  Afghan Restaurant   0.0


----Agincourt----
                        venue  freq
0                 Golf Course  0.50
1                Dance Studio  0.25
2  Construction & Landscaping  0.25
3           Afghan Restaurant  0.00
4                   Racetrack  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                venue  freq
0  Italian Restaurant   0.5
1        Skating Rink   0.5
2   Afghan Restaurant   0.0
3                Park   0.0
4           Racetrack   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                           venue  freq
0                    Pizza Place   0.5
1                  Grocery Store   0.5
2              Afghan Restaurant   0.0
3  Paper / Office Supplies Store   0.0
4                      Racetrac

                 venue  freq
0                Hotel  0.24
1           Restaurant  0.14
2          Coffee Shop  0.10
3           Hobby Shop  0.10
4  American Restaurant  0.10


----Emery,Humberlea----
                venue  freq
0       Grocery Store  0.25
1                Bank  0.25
2      Clothing Store  0.25
3  Chinese Restaurant  0.25
4            Pharmacy  0.00


----Fairview,Henry Farm,Oriole----
               venue  freq
0       Skating Rink  0.25
1                Bar  0.25
2        Pizza Place  0.25
3        Gas Station  0.25
4  Afghan Restaurant  0.00


----First Canadian Place,Underground city----
               venue  freq
0     Baseball Field   0.4
1          Racetrack   0.2
2             Garden   0.2
3        Golf Course   0.2
4  Afghan Restaurant   0.0


----Flemingdon Park,Don Mills South----
                           venue  freq
0                          Hotel   1.0
1              Afghan Restaurant   0.0
2  Paper / Office Supplies Store   0.0
3                      Ra

                 venue  freq
0                Hotel   0.4
1     Airport Terminal   0.2
2          Men's Store   0.2
3  Rental Car Location   0.2
4               Lounge   0.0


----The Beaches West,India Bazaar----
                     venue  freq
0              Coffee Shop  0.50
1             Intersection  0.25
2  New American Restaurant  0.25
3                     Park  0.00
4                Racetrack  0.00


----The Danforth West,Riverdale----
                 venue  freq
0           Hobby Shop  0.25
1  Rental Car Location  0.12
2  American Restaurant  0.12
3          Men's Store  0.12
4                  Bar  0.12


----The Junction North,Runnymede----
               venue  freq
0        Coffee Shop  0.50
1   Airport Terminal  0.25
2            Brewery  0.25
3  Afghan Restaurant  0.00
4               Park  0.00


----The Kingsway,Montgomery Road,Old Mill North----
               venue  freq
0     Baseball Field   0.4
1          Racetrack   0.2
2        Coffee Shop   0.2
3        Gas 

<h3>Let's put that into a pandas dataframe</h3>

In [200]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [201]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Baseball Field,Garden,Golf Course,Racetrack,Vietnamese Restaurant,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant
1,Agincourt,Golf Course,Dance Studio,Construction & Landscaping,Garden,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Italian Restaurant,Skating Rink,Vietnamese Restaurant,Garden,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Pizza Place,Grocery Store,Vietnamese Restaurant,Garden,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court
4,"Alderwood,Long Branch",Bakery,Seafood Restaurant,Fast Food Restaurant,Korean Restaurant,Vietnamese Restaurant,Supermarket,Restaurant,Sandwich Place,Electronics Store,Pool Hall


<h2>Cluster Neighborhoods</h2>
<h6>Run k-means to cluster the neighborhood into 5 clusters</h6>

In [202]:
k=5
toronto_grouped_cluster = toronto_grouped.drop('Neighborhood',1)
kmeans = KMeans(n_clusters = k, random_state=0).fit(toronto_grouped_cluster)
kmeans.labels_

array([1, 1, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 4, 2, 3, 3, 3, 1, 4, 3, 2,
       3, 2, 1, 1, 0, 2, 2, 2, 1, 3, 3, 4, 3, 3, 2, 2, 2, 3, 3, 1, 2, 3,
       2, 3, 3, 1, 4, 3, 3, 4, 3, 3, 3, 3, 4, 2, 3, 3, 3, 2, 1, 3, 2, 3,
       2, 0, 1, 3, 3, 2, 2, 1, 3, 3, 3, 1, 1, 3, 4, 2, 4, 2, 4, 1, 0, 3,
       2, 3, 0, 3, 3, 0, 3, 3])

<h6>Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.</h6>

In [204]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

toronto_merge = toronto_df_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merge = toronto_merge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merge.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,3.0,Italian Restaurant,Dessert Shop,Bakery,Shopping Mall,Grocery Store,Vietnamese Restaurant,Garden,Discount Store,Drugstore,Electronics Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,3.0,Moving Target,Vietnamese Restaurant,Gas Station,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Mountain,Vietnamese Restaurant,Gas Station,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3.0,Mountain,Vietnamese Restaurant,Gas Station,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court,Food Stand


In [211]:
toronto_merge=toronto_merge.dropna()

In [212]:
toronto_merge['Cluster_Labels'] = toronto_merge.Cluster_Labels.astype(int)

In [214]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merge['Latitude'], toronto_merge['Longitude'], toronto_merge['Neighborhood'], toronto_merge['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2>Examine Clusters</h2>

<h3>Cluster 1</h3>

In [217]:
toronto_merge.loc[toronto_merge['Cluster_Labels'] == 0, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,East York,0,Rental Car Location,Drugstore,IT Services,Department Store,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
39,East York,0,Rental Car Location,Construction & Landscaping,Drugstore,Deli / Bodega,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
47,Central Toronto,0,Home Service,Drugstore,Lounge,Vietnamese Restaurant,Gas Station,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Food Court
98,York,0,Rental Car Location,Drugstore,IT Services,Department Store,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
102,Etobicoke,0,Rental Car Location,Drugstore,IT Services,Department Store,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Food Court,Food Stand


<h3>Cluster 2</h3>

In [218]:
toronto_merge.loc[toronto_merge['Cluster_Labels'] == 1, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,1,Golf Course,Dance Studio,Construction & Landscaping,Garden,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court
53,Downtown Toronto,1,Baseball Field,Coffee Shop,Racetrack,Gas Station,Vietnamese Restaurant,Garden,Dessert Shop,Discount Store,Drugstore,Electronics Store
55,Downtown Toronto,1,Baseball Field,Garden,Golf Course,Racetrack,Vietnamese Restaurant,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant
58,Downtown Toronto,1,Baseball Field,Garden,Golf Course,Racetrack,Vietnamese Restaurant,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant
60,Downtown Toronto,1,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Golf Course,Racetrack,Vietnamese Restaurant,Food Stand,Department Store,Dessert Shop,Discount Store,Drugstore
61,Downtown Toronto,1,Baseball Field,Garden,Golf Course,Racetrack,Vietnamese Restaurant,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant
67,Downtown Toronto,1,Baseball Field,Coffee Shop,Racetrack,Gas Station,Vietnamese Restaurant,Garden,Dessert Shop,Discount Store,Drugstore,Electronics Store
69,Downtown Toronto,1,Gym / Fitness Center,Golf Course,Vietnamese Restaurant,Garden,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court
70,Downtown Toronto,1,Baseball Field,Garden,Golf Course,Racetrack,Vietnamese Restaurant,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant
77,West Toronto,1,Baseball Field,Garden,Golf Course,Racetrack,Vietnamese Restaurant,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant


<h3>Cluster 3</h3>

In [219]:
toronto_merge.loc[toronto_merge['Cluster_Labels'] == 2, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,2,Hotel,Vietnamese Restaurant,Gas Station,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
7,Scarborough,2,Afghan Restaurant,Coffee Shop,Hotel,Rental Car Location,BBQ Joint,Bakery,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant
8,Scarborough,2,Hotel,Racecourse,Gym / Fitness Center,Rental Car Location,Pizza Place,Coffee Shop,Swiss Restaurant,Mediterranean Restaurant,Chinese Restaurant,Department Store
9,Scarborough,2,American Restaurant,Hotel,Paper / Office Supplies Store,Vietnamese Restaurant,Gas Station,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court
27,North York,2,Hotel,Vietnamese Restaurant,Gas Station,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
34,North York,2,Hotel,Vietnamese Restaurant,Gas Station,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
37,East Toronto,2,Hotel,Rental Car Location,Airport Terminal,Men's Store,Garden,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant
38,East York,2,Rental Car Location,Hotel,Garden,Department Store,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court
40,East York,2,Hotel,Restaurant,American Restaurant,Hobby Shop,Coffee Shop,Italian Restaurant,Sandwich Place,Steakhouse,Taxi,Café
41,East Toronto,2,Hobby Shop,Rental Car Location,Hotel,American Restaurant,Bar,Coffee Shop,Men's Store,Garden,Discount Store,Drugstore


<h3>Cluster 4</h3>

In [220]:
toronto_merge.loc[toronto_merge['Cluster_Labels'] == 3, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,Italian Restaurant,Dessert Shop,Bakery,Shopping Mall,Grocery Store,Vietnamese Restaurant,Garden,Discount Store,Drugstore,Electronics Store
2,Scarborough,3,Moving Target,Vietnamese Restaurant,Gas Station,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
3,Scarborough,3,Mountain,Vietnamese Restaurant,Gas Station,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
4,Scarborough,3,Mountain,Vietnamese Restaurant,Gas Station,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
5,Scarborough,3,Sandwich Place,Discount Store,Coffee Shop,Grocery Store,Pizza Place,Vietnamese Restaurant,Food Stand,Dessert Shop,Drugstore,Electronics Store
10,Scarborough,3,BBQ Joint,Bank,Gas Station,Vietnamese Restaurant,Gift Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court
11,Scarborough,3,Convenience Store,Vietnamese Restaurant,Deli / Bodega,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
13,Scarborough,3,Italian Restaurant,Gas Station,Dessert Shop,Bakery,Shopping Mall,Grocery Store,Skating Rink,Food Stand,Discount Store,Drugstore
14,Scarborough,3,Italian Restaurant,Skating Rink,Vietnamese Restaurant,Garden,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court
15,Scarborough,3,Dance Studio,Italian Restaurant,Construction & Landscaping,Vietnamese Restaurant,Gas Station,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court


<h3>Cluster 5</h3>

In [221]:
toronto_merge.loc[toronto_merge['Cluster_Labels'] == 4, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,4,Coffee Shop,Vietnamese Restaurant,Deli / Bodega,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
42,East Toronto,4,Coffee Shop,New American Restaurant,Intersection,Vietnamese Restaurant,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
51,Downtown Toronto,4,Coffee Shop,Intersection,Café,Vietnamese Restaurant,Gas Station,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court
59,Downtown Toronto,4,Home Service,Coffee Shop,Tennis Court,Vietnamese Restaurant,Garden,Dessert Shop,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant
65,Central Toronto,4,Coffee Shop,Airport Terminal,Brewery,Vietnamese Restaurant,Gas Station,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court
75,Downtown Toronto,4,Coffee Shop,Intersection,Pharmacy,Vietnamese Restaurant,Garden,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court
76,West Toronto,4,Coffee Shop,New American Restaurant,Intersection,Vietnamese Restaurant,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court,Food Stand
81,York,4,Coffee Shop,Airport Terminal,Brewery,Vietnamese Restaurant,Gas Station,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court
93,Etobicoke,4,Coffee Shop,Intersection,Café,Vietnamese Restaurant,Gas Station,Discount Store,Drugstore,Electronics Store,Fast Food Restaurant,Food Court
